In [1]:
import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
import spacy
from spacy.matcher import Matcher 
from spacy.tokens import Span 
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import spacy
import textacy

In [2]:
nlp=spacy.load('en_core_web_sm')
pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [3]:
dataframe = pd.read_csv("dataSet.csv")

In [4]:
dataframe

,text,sentiment,id
0,missed sam spin tonight stooped work boo,0,1883595516
1,bad news,0,1883595722
2,blackberry broken feeling withdrawal get fixed no leash,0,1883595996
3,work feel sick,0,1883596562
4,morning twittertown valencia since discovered someone fridge seriously good,0,1883596891
...,...,...,...
1589242,woke no school best feeling ever,4,2193601966
1589243,theda com cool hear old walt interview beta,4,2193601969
1589244,ready mojo makeover ask detail,4,2193601991
1589245,happy th birthday boo time tupac amaru shaker,4,2193602064


In [5]:
candidate_sentences = dataframe['text']

In [6]:
candidate_sentences.shape


(1589247,)

In [7]:
candidate_sentences.sample(5)


110349                                                 people leave dog hot car mean
731843                                                                    good night
1046075    hey know brother bought boxer word glow dark awesome fly whole lot better
1435705                                               drew big picture computer desk
1261603                                               yea bet cannon wait see auburn
Name: text, dtype: object

In [8]:
'''def get_entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  #############################################################
  
  for tok in nlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]
'''

'def get_entities(sent):\n  ## chunk 1\n  ent1 = ""\n  ent2 = ""\n\n  prv_tok_dep = ""    # dependency tag of previous token in the sentence\n  prv_tok_text = ""   # previous token in the sentence\n\n  prefix = ""\n  modifier = ""\n\n  #############################################################\n  \n  for tok in nlp(sent):\n    ## chunk 2\n    # if token is a punctuation mark then move on to the next token\n    if tok.dep_ != "punct":\n      # check: token is a compound word or not\n      if tok.dep_ == "compound":\n        prefix = tok.text\n        # if the previous word was also a \'compound\' then add the current word to it\n        if prv_tok_dep == "compound":\n          prefix = prv_tok_text + " "+ tok.text\n      \n      # check: token is a modifier or not\n      if tok.dep_.endswith("mod") == True:\n        modifier = tok.text\n        # if the previous word was also a \'compound\' then add the current word to it\n        if prv_tok_dep == "compound":\n          modifier = p

In [9]:
'''def get_relation(sent):

  doc = nlp(sent)

  # Matcher class object 
  matcher = Matcher(nlp.vocab)

  #define the pattern 
  pattern = [{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}] 

  matcher.add("matching_1", None, pattern) 

  matches = matcher(doc)
  k = len(matches) - 1

  span = doc[matches[k][1]:matches[k][2]] 

  return(span.text)
  
'''

'def get_relation(sent):\n\n  doc = nlp(sent)\n\n  # Matcher class object \n  matcher = Matcher(nlp.vocab)\n\n  #define the pattern \n  pattern = [{\'DEP\':\'ROOT\'}, \n            {\'DEP\':\'prep\',\'OP\':"?"},\n            {\'DEP\':\'agent\',\'OP\':"?"},  \n            {\'POS\':\'ADJ\',\'OP\':"?"}] \n\n  matcher.add("matching_1", None, pattern) \n\n  matches = matcher(doc)\n  k = len(matches) - 1\n\n  span = doc[matches[k][1]:matches[k][2]] \n\n  return(span.text)\n  \n'

In [10]:
dataframe = shuffle(dataframe)
dataframe.reset_index(inplace=True, drop=True) 

train, test = train_test_split(dataframe, test_size=0.2)
train_1, train_2 = train_test_split(train, test_size=0.5)


In [11]:
mask = train_1['sentiment']==0
df_train_1_negative = train_1[mask]

In [12]:
mask2 = train_2['sentiment']==0
df_train_2_negative = train_2[mask2]

In [13]:
df_train_1_negative

,text,sentiment,id
1063692,tell truth no not sub contractor licence goto school,0,2321968368
1417081,thanks no open wife near though,0,1979994289
1261855,miss school,0,2264414319
1208359,sound barrier broke engine,0,1988855239
959818,milling dean believed last survivor titanic died,0,1989928687
...,...,...,...
856047,missed smee snuggling blackberry sleep well,0,1988907581
842491,year long yet seems like yesterday,0,2004916877
704461,hug,0,2253907979
828616,take lunch sizzler child like boring,0,1978625857


In [14]:
df_train_1_positive = train_1[~mask]

In [15]:
df_train_2_positive = train_2[~mask2]

In [16]:
df_train_1_positive

,text,sentiment,id
268052,not going fail babe finished tomorrow afternoon worry putting pic fb,4,2063742128
409117,good morning,4,1695342983
406886,morning,4,1753582745
1214716,going dinner bonefish,4,1964670367
298952,welcome,4,2052396592
...,...,...,...
1126863,tracy sleep good night goodnight,4,2176384620
623320,vibrant aha run faster,4,2012311570
473464,hold razor blade hand extra effect,4,2058982572
369317,thank loving,4,2060241499


In [17]:
candidate_sentences_train_1_positive = df_train_1_positive['text']

In [18]:
candidate_sentences_train_1_positive

268052     not going fail babe finished tomorrow afternoon worry putting pic fb
409117                                                             good morning
406886                                                                  morning
1214716                                                   going dinner bonefish
298952                                                                  welcome
                                           ...                                 
1126863                                        tracy sleep good night goodnight
623320                                                   vibrant aha run faster
473464                                       hold razor blade hand extra effect
369317                                                             thank loving
992761                                                             see two soon
Name: text, Length: 317008, dtype: object

In [19]:
triples_pos = []
nlp = spacy.load('en_core_web_sm')
for i in tqdm(candidate_sentences_train_1_positive):
    text = nlp(i)
    text_ext = textacy.extract.subject_verb_object_triples(text)
    triples_pos.append(list(text_ext))

100%|██████████| 317008/317008 [1:16:11<00:00, 69.34it/s]


In [20]:
_triples_pos = []
for i in triples_pos:
    if len(i) > 0:
        _triples_pos.append(i)
    

In [21]:
len(_triples_pos)

73320

In [22]:
_triples_pos

[[(fail babe, putting, fb), (afternoon worry, putting, fb)],
 [(get, add, everyone), (get, add, train pay vip)],
 [(talk ya, going, bed)],
 [(one, paying, attention)],
 [(decision, go, see)],
 [(lettuce, grow, type window)],
 [(university, want, party)],
 [(hrhrhr, try put, slr machine)],
 [(homework list, not forget, lol)],
 [(aha love, sitting, ft)],
 [(mash dinner, produced, germany)],
 [(camel hill, give, plankton), (camel hill, give, hilarious)],
 [(sunday bay, make, jose)],
 [(love sight, going, background)],
 [(reward, may download, copy)],
 [(cheer, would create, charity tyre)],
 [(stuff, keep, em), (stuff, keep, coming)],
 [(time, hit, shop)],
 [(competitor, turn, opportunity)],
 [(aww, not know, week end)],
 [(music, break, thing)],
 [(someone, help, sale)],
 [(maggie, make, simpson)],
 [(rascal, started, spraying), (went, requires, info)],
 [(sweeting, would never known, one)],
 [(time hun, getting, bed)],
 [(go, tried, film)],
 [(nice, came, station)],
 [(sleep cause, hate,

In [23]:
# extract subject
source_pos = [i[0][0] for i in _triples_pos]


In [24]:
# extract object
target_pos = [i[0][2] for i in _triples_pos]

In [25]:
relations_pos = [i[0][1] for i in _triples_pos]

In [26]:
#create the graph
positive_class_knowledge_graph = pd.DataFrame({'source_pos':source_pos, 'target_pos':target_pos, 'edge_pos':relations_pos})

In [27]:
G_pos = nx.from_pandas_edgelist(positive_class_knowledge_graph, "source_pos", "target_pos", edge_attr=True, create_using=nx.MultiDiGraph())

In [28]:
#TO PLOT 

#plt.figure(figsize=(12,12))
#pos = nx.spring_layout(G)
#nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
#plt.show()

In [29]:
positive_class_knowledge_graph

,source_pos,target_pos,edge_pos
0,"(fail, babe)",(fb),(putting)
1,(get),(everyone),(add)
2,"(talk, ya)",(bed),(going)
3,(one),(attention),(paying)
4,(decision),(see),(go)
...,...,...,...
73315,"(lease, te)","(stuff, work)",(moving)
73316,"(aww, thanks)",(anything),(talk)
73317,(video),(bos),(posted)
73318,(takin),(bahamas),(read)


In [30]:
positive_class_knowledge_graph.edge_pos.value_counts()

(made)            4
(lost)            4
(need)            4
(told)            3
(got)             3
                 ..
(should, able)    1
(go)              1
(give)            1
(thank)           1
(decided)         1
Name: edge_pos, Length: 71830, dtype: int64

In [31]:
G_pos

In [32]:
# THE SAME TREATMENT FOR NEGATIVE GRAPH

In [33]:
candidate_sentences_train_1_negative = df_train_1_negative['text']

In [116]:
positive_class_knowledge_graph

,source_pos,target_pos,edge_pos
0,"(fail, babe)",(fb),(putting)
1,(get),(everyone),(add)
2,"(talk, ya)",(bed),(going)
3,(one),(attention),(paying)
4,(decision),(see),(go)
...,...,...,...
73315,"(lease, te)","(stuff, work)",(moving)
73316,"(aww, thanks)",(anything),(talk)
73317,(video),(bos),(posted)
73318,(takin),(bahamas),(read)


In [34]:
triples_neg = []
nlp = spacy.load('en_core_web_sm')
for i in tqdm(candidate_sentences_train_1_negative):
    text = nlp(i)
    text_ext = textacy.extract.subject_verb_object_triples(text)
    triples_neg.append(list(text_ext))
    

100%|██████████| 318690/318690 [1:21:53<00:00, 64.85it/s] 


In [35]:
_triples_neg = []
for i in triples_neg:
    if len(i) > 0:
        _triples_neg.append(i)

In [36]:
# extract subject
source_negatives = [i[0][0] for i in _triples_neg]

In [37]:
# extract object
target_negatives = [i[0][2] for i in _triples_neg]

In [38]:
relations_negatives = [i[0][1] for i in _triples_neg]

In [39]:
#create the graph
negative_class_knowledge_graph = pd.DataFrame({'source_neg':source_negatives, 'target_neg':target_negatives, 'edge_neg':relations_negatives})

In [40]:
negative_class_knowledge_graph.edge_neg.value_counts()

(wish)                4
(lost)                3
(used)                3
(find)                3
(stop)                3
                     ..
(live)                1
(want)                1
(hurt)                1
(could, not, make)    1
(say)                 1
Name: edge_neg, Length: 83177, dtype: int64

In [41]:
G_neg = nx.from_pandas_edgelist(negative_class_knowledge_graph, "source_neg", "target_neg", edge_attr=True, create_using=nx.MultiDiGraph())

In [42]:
def mu(edge, relations):
    if edge in relations:
        return 1
    else:
        return 0

In [43]:
len(G_pos)

146640

In [44]:
len(G_neg)

169714

In [45]:
def computeCS(G1, G2, relationsG1, relationsG2):
    totalSumMu = 0
    globalSize = min(len(G1), len(G2))
    for i in relationsG1:
        totalSumMu += mu(i, relationsG2)
    vMin = min(len(G1), len(G2))
    return totalSumMu/vMin
        

In [46]:
computeCS(G_pos, G_neg, relations_pos, relations_negatives)

0.498936170212766

In [104]:
def commonNodes(G1, G2):
    percentages = []
    total = 0
    for i in G1:
        total = 0
        for k in G2:
            if i == k:
                total += 1
            else:
                total += 0
        percentages.append(total/len(G2))
 
    return percentages
    

In [8]:
def commonNode(node_lists, graph_class_list):
    belongs_list = []
    for i in node_lists:
        if i in graph_class_list:
            belongs_list.append(1)
            
        else:
            
            belongs_list.append(0)
            
    return belongs_list

In [9]:
def commonEdges(relationsG1, relationsG2):
    percentages = []
    total = 0
    for i in relationsG1:
        total = 0
        for k in relationsG2:
            if i == k:
                total += 1
            else:
                total += 0
        percentages.append(total/len(relationsG2))
 
    return percentages

In [49]:
def MCS(G1, G2): #Maximum Common Subgraph
    return commonNodes(G1, G2)/min(len(G1), len(G2))

In [50]:
print(MCS(G_pos, G_neg))

0.997360883797054


In [51]:
print(commonNodes(G_pos, G_neg))

146253


In [52]:
def MCSUES(G1, G2): #Total number of edges contained in the MCS
    return commonEdges(G1, G2)/min(len(G1), len(G2))

In [53]:
train_2

,text,sentiment,id
984079,funny salad mm get hit lot wacky lover,4,1986980483
454618,lot fun hanging dung park today,4,1956529095
1312073,aww wanted spend time aunt mao dad though,0,2260887859
157533,appears bypassed storm yet hear thunder should able rain,0,2066603484
666838,sick stomach worse yesterday,0,2192492984
...,...,...,...
1304626,wishing good new start upcoming week,4,2068022183
66748,bubbletweet not work poppy trying break hear people talking save u,0,2009369158
839218,dude totally promoted fund manager time taking job,4,1965545008
770147,sitting system architect training deep odd anti delegate,0,2001902772


In [54]:
#df_train_2_positive
#df_train_2_negative

In [55]:
candidate_sentences_train_2_positive = df_train_2_positive['text'] #label 4

In [56]:
candidate_sentences_train_2_negative = df_train_2_negative['text']

In [57]:
triples_set_2_pos = []
nlp = spacy.load('en_core_web_sm')
for i in tqdm(candidate_sentences_train_2_positive):
    text = nlp(i)
    text_ext = textacy.extract.subject_verb_object_triples(text)
    triples_set_2_pos.append(list(text_ext))

100%|██████████| 316971/316971 [1:07:26<00:00, 78.33it/s] 


In [58]:
triples_set_2_neg = []
nlp = spacy.load('en_core_web_sm')
for i in tqdm(candidate_sentences_train_2_negative):
    text = nlp(i)
    text_ext = textacy.extract.subject_verb_object_triples(text)
    triples_set_2_neg.append(list(text_ext))

100%|██████████| 318728/318728 [55:18<00:00, 96.03it/s]  


In [59]:
_triples_set_2_pos = []
for i in triples_set_2_pos:
    if len(i) > 0:
        _triples_set_2_pos.append(i)
        


In [60]:
_triples_set_2_neg = []
for i in triples_set_2_neg:
    if len(i) > 0:
        _triples_set_2_neg.append(i)

In [61]:
label_positive = [4]*(len(_triples_set_2_pos))
label_negative = [0]*(len(_triples_set_2_neg))


In [62]:
_triples_set_2_pos

[[(mm, get hit, lover)],
 [(message ko, say, webster)],
 [(anxiety, damned, bite)],
 [(chat show, come, beige)],
 [(people poker scene, got, ta keep)],
 [(mao, scared, flirt)],
 [(cannon wait, see, guy)],
 [(yay, got, way), (lauren, got, way)],
 [(screw, try, sleep)],
 [(australia, could chill, david)],
 [(yay, rescued, leon)],
 [(woke, go, town)],
 [(thanks hug, made, morning)],
 [(rock, got upgraded, row balcony)],
 [(day cannon help, wish, prom)],
 [(drinker, huh not drink, form coffee)],
 [(woke, figured, break)],
 [(dhellohannah, follow, need)],
 [(someone, say, got), (demo cd play, ordered, simms)],
 [(diversity, got, sympathy vote)],
 [(fact, seem, cloaking device)],
 [(song, not forget, watch)],
 [(cock eye, see, flimsy box)],
 [(lol love, should play, challenge)],
 [(dog weekend, crashed, plan)],
 [(dude, should footer, link logo)],
 [(undefined, meant, performing)],
 [(porn, passed, cock)],
 [(command, put, shirt)],
 [(offense, taken, family)],
 [(summer, get, touch), (kelley

In [63]:
# Calling DataFrame constructor on list 
df1 = pd.DataFrame(_triples_set_2_pos) 
df1["label"] = label_positive

In [64]:
df1

,0,1,2,3,4,5,label
0,"((mm), (get, hit), (lover))",None,None,None,None,None,4
1,"((message, ko), (say), (webster))",None,None,None,None,None,4
2,"((anxiety), (damned), (bite))",None,None,None,None,None,4
3,"((chat, show), (come), (beige))",None,None,None,None,None,4
4,"((people, poker, scene), (got), (ta, keep))",None,None,None,None,None,4
...,...,...,...,...,...,...,...
73977,"((yeh), (not, think), (happen))",None,None,None,None,None,4
73978,"((option), (not, deserve), (rant))",None,None,None,None,None,4
73979,"((almond, chocolate, cake), (put), (making))",None,None,None,None,None,4
73980,"((working), (got, caught), (link))",None,None,None,None,None,4


In [65]:
df2 = pd.DataFrame(_triples_set_2_neg) 
df2["label"] = label_negative

In [87]:
frames = [df1, df2]

In [88]:
print(len(df1))
print(len(df2))

73982
84195


In [101]:
result = pd.concat(frames)

In [102]:
result

,0,1,2,3,4,5,label
0,"((mm), (get, hit), (lover))",None,None,None,None,None,4
1,"((message, ko), (say), (webster))",None,None,None,None,None,4
2,"((anxiety), (damned), (bite))",None,None,None,None,None,4
3,"((chat, show), (come), (beige))",None,None,None,None,None,4
4,"((people, poker, scene), (got), (ta, keep))",None,None,None,None,None,4
...,...,...,...,...,...,...,...
84190,"((mao), (love), (everyone))",None,None,None,None,None,0
84191,"((wish), (could, put), (not, live))","((wish), (could, put), (u))",None,None,None,None,0
84192,"((people), (keep), (un))","((people), (keep), (following))",None,None,None,None,0
84193,"((space), (left), (thing))",None,None,None,None,None,0


In [72]:
G_pos

In [73]:
result.to_csv("/home/learnsity/Documents/usbSentimentAnalysis/sentimentProy/sentiment/result.csv")

In [74]:
negative_class_knowledge_graph.to_csv("/home/learnsity/Documents/usbSentimentAnalysis/sentimentProy/sentiment/negative_class.csv")

In [75]:
positive_class_knowledge_graph.to_csv("/home/learnsity/Documents/usbSentimentAnalysis/sentimentProy/sentiment/positive_class.csv")

In [41]:
result = pd.read_csv("/home/learnsity/Documents/usbSentimentAnalysis/sentimentProy/sentiment/result.csv")

In [42]:
negative_class_knowledge_graph = pd.read_csv("/home/learnsity/Documents/usbSentimentAnalysis/sentimentProy/sentiment/negative_class.csv")

In [43]:
positive_class_knowledge_graph = pd.read_csv("/home/learnsity/Documents/usbSentimentAnalysis/sentimentProy/sentiment/positive_class.csv")

In [44]:
result

,Unnamed: 0,0,1,2,3,4,5,label
0,0,"(mm, get hit, lover)",NaN,NaN,NaN,NaN,NaN,4
1,1,"(message ko, say, webster)",NaN,NaN,NaN,NaN,NaN,4
2,2,"(anxiety, damned, bite)",NaN,NaN,NaN,NaN,NaN,4
3,3,"(chat show, come, beige)",NaN,NaN,NaN,NaN,NaN,4
4,4,"(people poker scene, got, ta keep)",NaN,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...
158172,84190,"(mao, love, everyone)",NaN,NaN,NaN,NaN,NaN,0
158173,84191,"(wish, could put, not live)","(wish, could put, u)",NaN,NaN,NaN,NaN,0
158174,84192,"(people, keep, un)","(people, keep, following)",NaN,NaN,NaN,NaN,0
158175,84193,"(space, left, thing)",NaN,NaN,NaN,NaN,NaN,0


In [45]:
result = result[['0','label']]


In [49]:
result['0'][1].split(",")[1]

' say'

In [120]:
def percentageOfNode(node, listOfNodes):
    percentage = 0
    for i in listOfNodes:
        
        if i == node:
            percentage += percentage
    return percentage/len(listOfNodes)

In [52]:
for i in result['0']:
    print(i.split(",")[1])
    

 get hit
 say
 damned
 come
 got
 scared
 see
 got
 try
 could chill
 rescued
 go
 made
 got upgraded
 wish
 huh not drink
 figured
 follow
 say
 got
 seem
 not forget
 see
 should play
 crashed
 should footer
 meant
 passed
 put
 taken
 get
 make
 discovered
 allowed
 see
 adjust
 ate
 sipping
 went
 keep
 shot
 playing
 should help
 relax
 recommends
 eat
 get
 rested
 agree
 got
 keep
 popped
 see
 died
 need
 care
 laid
 got
 keep
 enjoy
 put
 shipped
 love
 keep
 left
 see
 done
 appears
 deserve
 want
 would say
 need
 arrange
 missing
 awaiting
 get
 not become
 want
 could give
 never got
 stay
 called
 ve called
 try
 enjoying
 get finished
 come
 lol
 give
 managed
 supposed
 get
 cooked
 visit
 bought
 let
 put
 got
 see
 deserve
 feel
 get
 fought
 take
 show
 called
 must rummage
 said
 got
 hear
 began
 not used
 not cry
 posted
 guess
 guide
 teach
 could used
 see
 changed
 come
 make
 get
 think
 not talk
 realize
 made
 enjoy
 think
 staring
 left
 could taste
 differ

 got
 try
 saw
 see
 find
 stay
 engaged
 reflects
 might luck
 aired
 find
 got
 extend
 say
 would love
 get
 heard
 answer
 pay
 started
 want
 want buy
 go
 feel
 get
 love
 go
 bring
 cooked
 saving
 seemed
 could play
 hope
 thought
 may mistaken
 want
 never forced
 fly
 lost
 made
 buy
 made
 started
 add
 visited
 agrees
 got
 post
 give
 brought
 rose
 kicked
 take
 know
 ruin
 try
 appreciate
 might finish
 seize
 continues
 keep
 need
 not take
 got
 impounded
 send
 take
 appreciated
 reach
 excited
 appreciate
 look
 went
 deemed
 cooking
 restore
 listen
 need
 should look
 made
 try
 not fooled
 play
 love
 excited
 set
 put
 give
 feel
 add
 wait
 add
 never not want
 need
 would linger
 watch
 give
 got
 descends
 left
 check
 know
 got
 getting
 mean
 happening
 went
 went
 need
 named
 made
 keep
 come
 want
 could throw
 remember
 take
 got
 not understand
 appreciate
 add
 follow
 get
 enjoy
 thank
 got
 drink
 made
 got
 want
 need
 played
 keep
 not touch
 got
 

 get
 hit
 buy
 want
 got
 make
 beat
 enjoy
 update
 work
 watch
 seem
 not leave
 might use
 took
 got
 fly
 saw
 should left
 got
 face
 toured
 should start
 promise
 told
 speak
 got
 believe
 soundtrack
 made
 lost
 say
 get caught
 remember
 come
 sneak
 get
 watch
 believe
 creates
 add
 got go
 think
 denied
 feel
 send
 not forget
 wait
 put
 get
 not touched
 get
 come
 need
 learned
 share
 issued
 going
 think
 keep
 took
 put
 could sort
 should make
 thank
 know
 know
 get
 love
 could go not think
 learned
 buy
 buenos
 try
 get printed
 love
 put
 finishing
 fight
 ate
 want
 add
 look
 love
 go
 not stop
 get
 must said
 appreciate
 bring
 said
 got
 would make
 not serving
 watched
 liked
 start
 resist
 take
 not want
 got
 give
 never said
 percent
 get
 get
 keep
 knew
 enjoy
 wait
 got
 raging
 listen
 go
 want
 fly
 not work
 going
 meet
 answer
 go
 say
 play
 die
 go
 moving
 write
 make
 update
 would say
 follow
 want
 say
 never fails
 tell
 would not mind


 figured
 showed
 taking
 feel
 need
 tied
 eat
 perform
 enjoy
 feel
 look
 need
 twittering
 should go
 reverted
 look
 take
 assume
 figured
 stuck
 thank
 keep
 need
 sent
 said
 try
 should got
 congrats
 enjoy
 spread
 love
 meet
 used go
 say
 reminded
 get planned
 played
 hear
 enjoy
 may need
 got
 seems
 changed
 wait
 notified
 getting married
 win
 read
 go
 finished
 not fret
 keep
 prefer
 would never imagined
 prefer
 say
 shot
 not exactly want seen
 should not problem
 contend
 face
 loved
 save
 managed
 enabled
 follow
 not caught
 wiv
 spent
 read
 enjoy
 hit
 go
 try
 got
 must try
 striven
 go
 go
 learn
 hooked
 got
 give
 not mind
 see
 going
 bring
 prefer
 go
 see
 would not memorize
 could save
 help
 showered
 looked
 done
 lol
 never knew
 dude
 need
 should give
 get
 putting
 see
 got married
 ca played
 checking
 think
 lost
 try
 got
 write
 not want
 know
 stop
 see
 add
 thinking
 took
 not ringing
 blessing
 know
 receive
 liked
 hear
 leave
 got
 a

 put
 go
 heard
 asked
 hate
 looked
 featured
 see
 get
 put
 seems
 flash
 going send
 become
 should wear
 must think
 face
 needed
 ask
 would ref
 alienate
 performed
 think
 added
 see
 asked
 seems
 add
 want
 started
 damned
 continue
 would love
 go
 getting
 write
 lend
 spend
 wondered
 keep
 go
 go
 hear
 find
 attend
 go
 love
 enjoy
 leave
 take
 see
 using
 done
 get
 put
 listed
 responded
 posting
 look
 not sit
 beat
 click
 missed
 update
 came
 should start
 write
 want
 come
 need
 looked
 said
 love
 started
 would pleased
 liked
 need
 enjoyed
 not giving
 question
 tell
 not going
 recovering
 try
 hooked
 written not
 go
 done
 get fired
 need
 going
 never use
 seen
 explains
 continue
 wish
 got
 found
 congrats
 add
 get
 made
 want
 done
 go
 got
 got
 might take
 lost
 would mad not love
 tell
 continue
 think
 never cease
 want
 needed
 expect
 sat
 keep
 punish
 skinned
 allowing
 give
 come
 call
 spend
 said
 come
 keep
 not share
 mean
 knew
 expected

...

 causing
 got
 kept
 not going
 got
 got kicked
 dorkassed
 met
 spend
 tell
 burn
 started
 suck
 shut
 would go
 managed
 miss not love
 recruiting
 got lost
 make
 ate
 keep
 hang
 not published
 got
 not work
 seems
 hanging
 missing
 find
 stopped
 getting
 would kill
 headed
 die
 believe
 could arch
 want
 getting
 want
 took
 could snuggled
 drove
 suck
 cut
 love
 cry
 throw
 involves
 na become
 could reply
 not wake
 take
 hoped
 see
 killed
 start
 feel
 believe
 not worked
 disappeared
 get
 stop
 done
 received
 get
 see
 got
 would kick
 get
 could not find
 make
 not cure
 ran
 win
 feel
 stopped
 could go
 keep
 give
 done
 teach
 would say
 exhausted
 not work
 started
 may rest
 start
 take
 posing
 loved
 got
 destroyed
 left
 want
 slept
 see
 need
 screwed
 get caught
 landed
 could help
 made
 moved
 try not serve
 need put
 add
 never show
 hope
 should go
 go
 not think
 done
 wanted
 rash
 saying
 downgrading
 keep
 want
 stolen
 dipped
 cut
 hurt
 met
 wish
 

 drove
 feel
 not remember
 get
 suck
 decided
 want
 could not pick
 started
 got
 want
 drove
 not let
 feel
 hit
 lost
 broke
 put
 wear
 not know
 went
 tempted
 got
 identify
 went
 told
 get
 need
 think
 opened
 give
 want
 say
 cultivated
 break
 hear
 zapped
 not kicked
 forgot
 busted
 speak
 hurt
 owns
 d not want
 not going
 got
 know
 would stop
 lost
 said
 provided
 feeling
 sounded
 must go
 would make
 take
 need
 rescued
 drink
 feel
 crashed
 need
 trust
 go
 would stop
 wish
 say
 never win
 go
 stuck
 wearing
 hashing not made
 got
 sent
 want
 followed
 checking
 not send
 could see
 waiting
 would love
 fall
 get
 died
 said
 seems
 made
 agree
 keep
 ate
 enjoyed
 stay
 live
 need
 tell
 watch
 would hang
 going
 knew
 become
 going
 miss
 regurgitate
 enjoyed
 remember
 lost
 should put
 going
 paying
 not want
 lost
 want
 got
 could not mine
 got
 go
 go
 named
 feel
 want
 got
 bold
 put
 not play
 get lost
 keep
 broke
 got
 keep
 teach
 wait
 invited
 keep

 visit
 got
 stressed
 stuck
 ate
 want
 screwed
 seems
 happened
 boyfriend
 spent
 appear
 cut
 hit
 woken
 could not attend
 wait
 bombed
 turn
 bought
 spoke
 come
 cry
 not tell
 pass
 not sound
 updated not read
 fly
 gave
 pull
 would give
 wait
 fill
 should not buzzed
 pulled
 informing
 want
 forget
 could sing
 never mind
 related
 saw
 want
 got
 gutted
 increased
 try
 unwell
 used
 slip
 lost
 go
 felt
 find
 snoring
 hate
 ended
 diced
 broke
 bring
 win
 bring
 talked
 packed
 stop
 turn
 take
 bought
 would let
 treating
 tell
 make
 going
 worked
 get
 worked
 got
 could spend
 broke
 offer
 got
 sent
 not want
 left
 begin
 lost
 left
 not work
 get
 left
 appears
 breaking
 go
 not give
 saved
 turned
 see
 pinched
 left
 put
 saved
 came
 could see
 burned
 wear
 done
 not happening
 left
 said
 might complicate
 think
 should go
 suck
 go
 made
 asked
 got
 want
 need
 broke
 supposed
 wishing put
 feel
 send
 know
 got made
 make
 should listen
 should taken
 fin

 said
 told
 not win
 get shot
 order
 finish
 might give
 barked
 take
 could stuck
 lose
 ended
 making
 knew
 going
 said
 got
 knew
 let
 not may close
 tell
 find
 got
 know
 pic
 not fit
 going
 got
 would shut
 not lying
 fly
 might mean
 lost
 got
 left
 not turn
 broke
 ate
 not understand
 not got
 knew
 picked
 not seen
 need
 could buy
 decided
 going making
 keep
 put
 go
 deserted
 suck
 lost
 watched
 not played
 want go
 stuck
 could play
 not know
 leaked
 need
 came
 seem
 want
 get
 hurt
 see
 could watching
 want
 give
 would start
 say
 went
 not working
 thought
 make
 go
 hurt
 attached
 should take
 slept
 not work
 not want
 not buy
 could not make
 wanted
 miss
 went
 hurt
 eliminated
 feel
 keep
 find
 tried
 want
 bring
 said
 send
 hit
 not notice
 got
 keep
 gained
 wanted
 left
 get
 going
 replying
 oven
 tried
 not turn
 design
 take
 blow
 go
 take
 believe
 wanted
 want
 packed
 broke
 not read
 wetted
 believe
 not play
 need
 not want
 lost
 cut
 we

 told
 not feel
 made
 love
 could put
 keep
 left
 fixed


In [53]:
edge_metric_pos = []
node_metric_pos = []
subgraph_metric_pos = []

edge_metric_neg = []
node_metric_neg = []
subgraph_metric_neg = []

    

In [65]:
list_of_relations_result = []
for i in result['0']:
    list_of_relations_result.append(i.split(",")[1].split(" ")[1])


In [66]:
list_of_sourc_result = []
for i in result['0']:
    list_of_sourc_result.append(i.split(",")[0].split("(")[1])


In [70]:
list_of_obj_result = []
for i in result['0']:
    list_of_obj_result.append(i.split(",")[2].split(")")[0].split(" ")[1])

In [75]:
edge_metric_pos = commonEdges(list_of_relations_result, positive_class_knowledge_graph["edge_pos"])

In [105]:
source_node_metric_pos = commonNodes(list_of_sourc_result, positive_class_knowledge_graph["source_pos"].tolist())

In [106]:
source_node_metric_pos

[0.0008456082924168031,
 0.0,
 0.0,
 0.0,
 0.0,
 0.001104746317512275,
 0.005537370430987452,
 0.003641571194762684,
 2.727768685215494e-05,
 0.00015002727768685214,
 0.003641571194762684,
 0.0010638297872340426,
 0.0,
 0.00020458265139116204,
 0.0,
 1.363884342607747e-05,
 0.0010638297872340426,
 0.0,
 0.005400981996726678,
 0.0001773049645390071,
 0.0003273322422258593,
 0.0015548281505728314,
 0.0,
 6.819421713038734e-05,
 0.0,
 0.001486633933442444,
 0.0,
 2.727768685215494e-05,
 0.0,
 4.091653027823241e-05,
 0.0004773595199127114,
 0.00023186033824331697,
 0.00019094380796508457,
 0.0,
 0.008742498636115658,
 0.0,
 0.0,
 0.00019094380796508457,
 0.002795962902345881,
 0.0004091653027823241,
 0.0,
 0.0,
 0.00023186033824331697,
 0.0,
 0.0,
 0.0,
 0.0009274413529732679,
 6.819421713038734e-05,
 0.0029187124931805784,
 0.0017866884888161484,
 0.0015548281505728314,
 0.005878341516639389,
 0.005537370430987452,
 0.005400981996726678,
 0.0,
 0.005060010911074741,
 0.0,
 0.0,
 0.0074195

In [107]:
target_node_metric_pos = commonNodes(list_of_obj_result, positive_class_knowledge_graph["target_pos"].tolist())

In [78]:
edge_metric_neg = commonEdges(list_of_relations_result, negative_class_knowledge_graph["edge_neg"])

In [108]:
source_node_metric_neg = commonNodes(list_of_sourc_result, negative_class_knowledge_graph["source_neg"].tolist())

In [109]:
target_node_metric_neg = commonNodes(list_of_obj_result, negative_class_knowledge_graph["target_neg"].tolist())

In [94]:
import numpy as np

In [110]:
df_result = pd.DataFrame(
    {'M1': edge_metric_pos,
     'M2': source_node_metric_pos,
     'M3': target_node_metric_pos,
     'M4': edge_metric_neg,
     'M5': source_node_metric_neg,
     'M6': target_node_metric_neg,
     'label': result["label"]
    })

In [111]:
df_result

,M1,M2,M3,M4,M5,M6,label
0,0.023622,0.000846,0.000123,0.024170,0.000613,0.000059,4
1,0.010747,0.000000,0.000341,0.007955,0.000000,0.000295,4
2,0.000136,0.000000,0.000109,0.000130,0.000035,0.000118,4
3,0.008333,0.000000,0.000000,0.005704,0.000000,0.000012,4
4,0.043999,0.000000,0.000109,0.044534,0.000000,0.000035,4
...,...,...,...,...,...,...,...
158172,0.010038,0.001105,0.011143,0.002793,0.000872,0.001367,0
158173,0.000000,0.002550,0.000000,0.000000,0.011549,0.000000,0
158174,0.014430,0.009083,0.000041,0.011867,0.009145,0.000047,0
158175,0.006110,0.000341,0.005060,0.011502,0.000247,0.004195,0


In [112]:
df_result.to_csv("/home/learnsity/Documents/usbSentimentAnalysis/sentimentProy/sentiment/df_result.csv")

In [113]:
data = pd.read_csv("/home/learnsity/Documents/usbSentimentAnalysis/sentimentProy/sentiment/df_result.csv")

In [114]:
data

,Unnamed: 0,M1,M2,M3,M4,M5,M6,label
0,0,0.023622,0.000846,0.000123,0.024170,0.000613,0.000059,4
1,1,0.010747,0.000000,0.000341,0.007955,0.000000,0.000295,4
2,2,0.000136,0.000000,0.000109,0.000130,0.000035,0.000118,4
3,3,0.008333,0.000000,0.000000,0.005704,0.000000,0.000012,4
4,4,0.043999,0.000000,0.000109,0.044534,0.000000,0.000035,4
...,...,...,...,...,...,...,...,...
158172,158172,0.010038,0.001105,0.011143,0.002793,0.000872,0.001367,0
158173,158173,0.000000,0.002550,0.000000,0.000000,0.011549,0.000000,0
158174,158174,0.014430,0.009083,0.000041,0.011867,0.009145,0.000047,0
158175,158175,0.006110,0.000341,0.005060,0.011502,0.000247,0.004195,0
